# Stream Ciphers

## 2.1 Introduction

Symmetric cryptography is split into **stream ciphers** and **block ciphers**.

- **Stream ciphers** encrypt bits individually. This is achieved by adding a bit from a key stream to a plaintext bit.
  - **Synchronous stream ciphers**: the key stream depends only on the key
  - **Asynchronous stream ciphers**: the key stream also depends on the ciphertext

- **Block ciphers** encrypt an entire block of plaintext bits at a time with the same key. This means that the encryption of any plaintext bit in a given block depends on every other plaintext bit in the same block.

>**Stream Cipher Encryption and Decryption**  
The plaintext, the ciphertext and the key stream consist of individual bits,  
i.e., $x_i,y_i,s_i \in \{0, 1\}$.  
Encryption: $y_i = e_{s_i}(x_i) \equiv x_i + s_i \bmod 2$.  
Decryption: $x_i = d_{s_i}(y_i) \equiv y_i + s_i \bmod 2$.


- Encryption and decryption are the same function
- Modulo 2 addition is equivalent to XOR operation (perfectly balanced, invertible)
- The security of a stream cipher **completely depends on the key stream**.


## 2.2 Random Numbers and an Unbreakable Stream Cipher


- **True Random Number Generators(TRNG)** are based on physical processes. Examples include coin flipping, rolling of dice, semiconductor noise, clock jitter in digital circuits and radioactive decay.  
In cryptography, TRNGs are often needed for generating session keys.


- **(General) Pseudorandom Number Generators (PRNG)** generate sequences which are computed from an initial seed value. 
  - Often they are computed recursively in the following way:
$$ s_0 = seed $$
$$ s_{i+1} = f(s_i), i = 0,1,\cdots $$
  - *Linear congruential generator*:
$$ s_0 = seed $$
$$ s_{i+1} \equiv a \cdot s_i + b \bmod m, i = 0,1,\cdots $$
  - Good statistical properties, meaning their output approximates a sequence of true random numbers.

  
- **Cryptographically Secure Pseudorandom Number Generators (CSPRNG)**  
  - Unpredictable: Given $n$ consecutive bits of the key stream $s_i, s_{i+1},\cdots,s_{i+n-1}$, there is no polynomial time algorithm that can predict the next bit $s_{n+1}$ or any preceding bits $s_{i−1}, s_{i−2},\cdots$ with better than 50% chance of success.  

  
- **Unconditional Security**
  - A cryptosystem is unconditionally or information-theoretically secure if it cannot be broken even with infinite computational resources.
  
  
- **One-Time Pad (OTP)**  
  A stream cipher for which
  1. the key stream $ s_0, s_1, s_2, \cdots $ is generated by a true random number generator
  2. the key stream is only known to the legitimate communicating parties
  3. every key stream bit $s_i$ is only used once
    
  is called a one-time pad. The one-time pad is unconditionally secure.
  
  **We need one key bit for every bit of plaintext.**
  
  **OTPs are rarely used in practice.**
  

## 2.2.3 Towards Practical Stream Ciphers

All known practical crypto algorithms (stream ciphers, block ciphers, public-key algorithms) are not unconditionally secure. The best we can hope for is **computational security**.

> **Computational Security**
>
> A cryptosystem is computationally secure if the best known algorithm for breaking it requires at least $t$ operations.

### Building Key Streams from PRNGs

Let’s assume a PRNG based on the linear congruential generator:

$$ S_0 = seed $$
$$ S_{i+1} \equiv A \cdot S_i+B \bmod m \quad (i = 0,1,\cdots)$$

where we choose $m$ to be 100 bits long and $S_i, A, B \in {0,1,\cdots,m−1}$ 

The modulus $m$ is part of the encryption scheme and is publicly known. The secret key comprises the values (A,B) and possibly the seed $S_0$, each with a length of 100. That gives us a key length of 200 bit. 

Encryption:
$$y_i \equiv x_i + s_i \bmod 2$$
where $s_i$ are the bits of the binary representation of the PRNG output symbols $S_j$.  


Assume the attcker knows the first 300 bits of plaintext (file header information or he guesses part of the plaintext). Since he certainly knows the ciphertext, he can now compute the first 300 bits of key stream as:

$$s_i \equiv y_i + x_i \bmod m, i=1,2,\cdots,300$$

These 300 bits immediately give the first three output symbols of the PRNG: $S_1 = (s_1,\cdots,s_{100})$, $S_2 = (s_{101}, \cdots, s_{200})$ and $S_3 = (s_{201},\cdots, s_{300})$. The attacker can now generate two equations:
$$ S2 \equiv A \cdot S1 + B \bmod m $$
$$ S3 \equiv A \cdot S2 + B \bmod m $$

we can solve the system, yielding:
$$A \equiv (S_2−S_3)/(S_1−S_2) \bmod m$$
$$B \equiv S_2−S_1(S_2−S_3)/(S_1−S_2) \bmod m$$

In case $\gcd((S_1−S_2),m)) \neq 1$, we get multiple solutions. However, with a fourth piece of known plaintext the key can uniquely be detected in almost all cases.

In summary: if we know a few pieces of plaintext, we can compute the key and decrypt the entire ciphertext! This type of attack is why the notation of CSPRNG was invented.


### Building Key Streams from CSPRNGs

Pretty much all pseudorandom number generators that are used for applications outside cryptography are not cryptographically secure.


- Optimized for software implementation
- Optimized for hardware implementation
- Using block ciphers as building blocks

Currently it seems to be easier for scientists to design "secure" block ciphers than stream ciphers.

In [1]:
def prng(A, B, m, seed):
    s0 = seed
    while True:
        s1 = (A*s0 + B) % m
        s0 = s1
        yield s1

In [2]:
a = prng(0x12345678, 0x98765321, 0x27635463, 1)

In [3]:
print(hex(next(a)))
print(hex(next(a)))
print(hex(next(a)))
print(hex(next(a)))

0xd1d580d
0xfb694d4
0x90bfb67
0x1b628d69


## 2.3 Shift Register-Based Stream Ciphers

An elegant way of realizing long pseudorandom sequences is to use **linear feedback shift registers (LFSRs)**.

An LFSR consists of *clocked storage elements (flip-flops)* and a *feedback path*.

Example:
> Assuming the initial state bits $s_0, s_1, s_2$:
> $$s_{i+3} \equiv s_{i+1}+s_i \bmod 2$$
> where $i = 0,1,2, \cdots$

A mathematical description:

> * $m$ flip-flops
> * $m$ possible feedback locations, all combined by the XOR operation.
> * Whether a feedback path is active or not, is defined by the feedback coefficient $p_0, p_1, \cdots , p_{m−1}$:

> Let's assume the LFSR is initially loaded with the values $s_0, \cdots , s_{m−1}$.
> $$s_m \equiv s_{m−1} p_{m−1} + \cdots + s_1 p_1 + s_0 p_0 \bmod 2$$
> $$s_{i+m} \equiv \sum_{j=0}^{m−1} p_j \cdot s_{i+j} \bmod 2; \quad s_i, p_j \in \{0,1\}; \quad i = 0,1,2, \cdots$$

The maximum sequence length generated by an LFSR of degree $m$ is $2^m−1$.

Note that only certain configurations $(p_0,\cdots, p_{m−1})$ yield maximum length LFSRs.

A cryptosystem where the key bits only occur in linear relationships makes a highly insecure cipher.

Attack:  
If we use an LFSR as a stream cipher, the secret key k is the feedback coefficient vector $(p_{m−1},\cdots, p_1, p_0)$.  
Assume the attacker Oscar knows:
* Some plaintext and the corresponding ciphertext
* The degree $m$ of the LFSR (or try a large number of possible $m$ values)

Let the known plaintext be given by $x_0,x_1,\cdots,x_{2m−1}$ and the corresponding ciphertext by $y_0,y_1, \cdots,y_{2m−1}$. With these $2m$ pairs of plaintext and ciphertext bits, Oscar reconstructs the first $2m$ key stream bits:
$$s_i \equiv x_i + y_i \bmod 2$$

Generate $m$ equations for the first $m$ values of $i$:
$$s_{i+m} \equiv \sum_{j=0}^{m−1} p_j \cdot s_{i+j} \bmod 2; \quad s_i, p_j \in \{0,1\}; \quad i = 0,1,\cdots,m-1$$

$m$ linear equations in $m$ unknowns $p_0, p_1, \cdots, p_{m−1}$. 

Major consequences: **as soon as Oscar knows $2m$ output bits of an LFSR of degree $m$, the $p_i$ coefficients can exactly be constructed by merely solving a system of linear equations.**

There are many stream ciphers which use *combinations* of several LFSRs to build strong cryptosystems.

**Trivium** is a relatively new stream cipher which uses an 80-bit key. It is based on a combination of three shift registers. Even though these are feedback shift registers, there are nonlinear components used to derive the output of each register, unlike the LFSRs that we studied in the previous section.

Almost all modern stream ciphers have two input parameters: a **key k** and an **initialization vector IV**.  
The IV serves as a randomizer and should take a new value for every encryption session. It is important to note that the IV does not have to be kept secret, it merely must change for every session. Such values are often referred to as *nonces*, which stands for “number used once”

In [4]:
hex(0xfffff >> 1)

'0x7ffff'

In [5]:
import gmpy2

def lfsr(S, P, m):
    while True:
        s1 = gmpy2.popcount(S & P) % 2
        S = (S >> 1) | (s1 << (m - 1))
        yield s1

In [6]:
k = lfsr(0x4, 0x3, 3)

In [7]:
for i in range(20):
    print(next(k))

0
1
1
1
0
0
1
0
1
1
1
0
0
1
0
1
1
1
0
0


## 2.5 Lessons Learned

* Stream ciphers are less popular than block ciphers in most domains such as Internet security. There are exceptions, for instance, the popular stream cipher RC4.

* Stream ciphers sometimes require fewer resources, e.g., code size or chip area, for implementation than block ciphers, and they are attractive for use in constrained environments such as cell phones.

* The requirements for a cryptographically secure pseudorandom number generator are far more demanding than the requirements for pseudorandom number generators used in other applications such as testing or simulation.

* The One-Time Pad is a provable secure symmetric cipher. However, it is highly impractical for most applications because the key length has to equal the message length.

* Single LFSRs make poor stream ciphers despite their good statistical properties. However, careful combinations of several LFSR can yield strong ciphers.